In [ ]:
from asbe.base import *
from asbe.models import *
from asbe.estimators import *
from econml.orf import DMLOrthoForest
from econml.dml import CausalForestDML
from causalml.inference.nn import CEVAE
from openbt.openbt import OPENBT
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from copy import deepcopy
import econml
from causalml.dataset import synthetic_data
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF

In [ ]:
import pymc3 as pm
from pymc3 import glm, Model, sample
import patsy

In [ ]:
with pm.Model() as model:
    # https://juanitorduz.github.io/glm_pymc3/
    family = pm.glm.families.Normal()
    data = pm.Data("data", ds["X_training"])
    labels = ["x"+str(i) for i in range(ds["X_training"].shape[1])]
    glm.GLM(y=ds["y_training"], x = data, family=family, labels=labels)
    trace = sample(3000, cores=2)  # draw 3000 posterior samples using NUTS sampling

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sd, x4, x3, x2, x1, x0, Intercept]


Sampling 2 chains for 1_000 tune and 3_000 draw iterations (2_000 + 6_000 draws total) took 24 seconds.


In [ ]:
pm.set_data({"data": ds["X_test"]}, model=model)
ppc_test = pm.sample_posterior_predictive(trace, model=model)

In [ ]:
ppc_test["y"].T, axis=1.shape

In [ ]:
np.random.seed(1005)
y, X, t, ite, _, e = synthetic_data(mode=1, n=1000, p=5, sigma=1.0)

In [ ]:
y1 = np.where(t == 1, y, y + ite)
y0 = np.where(t == 0, y, y - ite)
X_train, X_test, t_train, t_test, y_train, y_test, ite_train, ite_test, y1_train, y1_test, y0_train, y0_test = train_test_split(
    X, t, y, ite, y1, y0,  test_size=0.9, random_state=1005)
ds = {"X_training": X_train,
     "y_training": y_train,
     "t_training": t_train,
     "X_pool": deepcopy(X_test), 
     "y_pool": deepcopy(y_test),
     "t_pool": deepcopy(t_test),
     "y1_pool": y1_test,
     "y0_pool":y0_test,
     "X_test": X_test,
     "y_test": y_test,
      "t_test": t_test,
      "ite_test": ite_test
     }

In [ ]:
def test_acq(estimator, acq):
    asl = BaseActiveLearner(estimator = BaseITEEstimator(model = estimator,
                                         two_model=False,ps_model=None),
                        acquisition_function=acq,
                        assignment_function=MajorityAssignmentFunction(),
                        stopping_function = None,
                        dataset=ds)
    asl.fit()
    X_new, query_idx = asl.query(no_query=10)
    print(query_idx)
    asl.teach(query_idx)
    preds = asl.predict(asl.dataset["X_test"])
    print(asl.score())
    return True

def test_acq_ob(acq):
    asl = BaseActiveLearner(estimator = OPENBTITEEstimator(model = OPENBT(model="bart",ntrees=200),
                                         two_model=False,ps_model=None),
                        acquisition_function=acq,
                        assignment_function=MajorityAssignmentFunction(),
                        stopping_function = None,
                        dataset=ds)
    asl.fit()
    X_new, query_idx = asl.query(no_query=10)
    print(query_idx)
    asl.teach(query_idx)
    preds = asl.predict(asl.dataset["X_test"])
    print(asl.score())
    return True

def test_acq_gp(acq):
    asl = BaseActiveLearner(estimator = GPEstimator(model = GaussianProcessRegressor(ConstantKernel(1.0) * RBF()),
                                         two_model=False,
                                                    ps_model=None),
                        acquisition_function=acq,
                        assignment_function=MajorityAssignmentFunction(),
                        stopping_function = None,
                        dataset=ds)
    asl.fit()
    X_new, query_idx = asl.query(no_query=20)
    print(query_idx)
    asl.teach(query_idx)
    print(asl.dataset["X_training"].shape)
    preds = asl.predict(asl.dataset["X_test"])
    print(preds)
    print(asl.score())
    return True

In [ ]:
test_acq(RandomForestRegressor(), RandomAcquisitionFunction())

[624 272 519 511 540 152 627   0 324   3]
0.8009953751914765


True

In [ ]:
# test_acq(RandomForestRegressor(), UncertaintyAcquisitionFunction())

In [ ]:
test_acq_ob(UncertaintyAcquisitionFunction())

Overwriting k to agree with the model's default
Overwriting overallnu to agree with the model's default
Overwriting ntree to agree with the model's default
Overwriting ntreeh to agree with the model's default
Overwriting overallsd to agree with the model's default
Writing config file and data
/var/folders/44/gtm_t6x110jg6b13p4rbwkfh0000gn/T/openbtpy_rt9yaxdd
3+ x variables
Running model...
[ 24 834 782 432 597 761 691 860 671 230]
0.5682984748893779


True

In [ ]:
test_acq_ob(TypeSAcquistionFunction())

Overwriting k to agree with the model's default
Overwriting overallnu to agree with the model's default
Overwriting ntree to agree with the model's default
Overwriting ntreeh to agree with the model's default
Overwriting overallsd to agree with the model's default
Writing config file and data
/var/folders/44/gtm_t6x110jg6b13p4rbwkfh0000gn/T/openbtpy_20cu04y2
3+ x variables
Running model...
[415 277 122 201  13 185 356 144  75 278]
0.2804859827444472


True

In [ ]:
test_acq_ob(EMCMAcquisitionFunction(no_query=10, B=10))

Overwriting k to agree with the model's default
Overwriting overallnu to agree with the model's default
Overwriting ntree to agree with the model's default
Overwriting ntreeh to agree with the model's default
Overwriting overallsd to agree with the model's default
Writing config file and data
/var/folders/44/gtm_t6x110jg6b13p4rbwkfh0000gn/T/openbtpy_l5b308d9
3+ x variables
Running model...
[220 360 164 157 381 364 362 192 161 340]
0.2803380463534182


True

In [ ]:
test_acq_gp(RandomAcquisitionFunction())

[ 40 663 547 819  70 306 381  73 830 162 808  27 446 487 271 468 599 367
 246 761]
(120, 5)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
0.5337187351959323


True

In [ ]:
test_acq_gp(UncertaintyAcquisitionFunction())

[899 336 306 305 304 303 302 301 300 299 298 297 296 295 294 293 292 291
 290 289]
(120, 5)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
0.5337187351959323


True

In [ ]:
test_acq_gp(TypeSAcquistionFunction())

[899 336 306 305 304 303 302 301 300 299 298 297 296 295 294 293 292 291
 290 289]
(120, 5)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
0.5337187351959323


True

In [ ]:
test_acq_gp(EMCMAcquisitionFunction())

[ 74 899 282 307 306 305 304 303 302 301 300 299 298 297 296 295 294 293
 292 291]
(120, 5)
[[ 0.05700141  0.1801203   0.11909015 ...  0.12306411  0.06735509
   0.19608517]
 [ 4.21579622  4.56534956  4.38730066 ...  4.23056837  4.1342224
   4.25585053]
 [-1.45203465 -1.35548585 -1.42326089 ... -1.40811163 -1.43223976
  -1.44421973]
 ...
 [-2.8906508  -2.71975307 -2.75067577 ... -2.81802532 -2.75970871
  -2.75915661]
 [ 5.36241851  5.67510024  5.57619882 ...  5.59758598  5.53029302
   5.66760542]
 [ 2.00389031  1.99446016  2.07388858 ...  2.00301579  2.01332167
   2.05755454]]
3.670348490582053


True